In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from collections.abc import Iterable

In [20]:
def flatten(arr, exclusions = (str,)):
    new_arr = []
    for x in arr:
        if isinstance(x,Iterable) and not isinstance(x,exclusions):
            new_arr.extend(flatten(x))
        else:
            new_arr.append(x)
    return new_arr

In [5]:
isinstance([1,2,3],Iterable)

True

In [21]:
flatten([1,2,"cat",[4,5,[6,7]],9])

[1, 2, 'cat', 4, 5, 6, 7, 9]

In [19]:
flatten([1,2,"cat",[4,5,[6,7]],9])

RecursionError: maximum recursion depth exceeded in comparison